In [14]:
import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler

from tqdm.notebook import tqdm
import h5py
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

In [2]:
# Seed function
def set_seed(seed_val):
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed(seed_val)
    
set_seed(42)

In [3]:
batch_size = 64
epochs = 100
lr = 1e-4
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Build Model

In [4]:
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9, padding=2, padding_mode='replicate')
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=2, padding_mode='replicate')
        self.conv3 = nn.Conv2d(32, 1, kernel_size=5, padding=2, padding_mode='replicate')
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        
        x = self.conv3(x)
        
        return x

In [5]:
model = SRCNN()
model.to(device)
summary(model, (1, 33, 33))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 29, 29]           5,248
              ReLU-2           [-1, 64, 29, 29]               0
            Conv2d-3           [-1, 32, 33, 33]           2,080
              ReLU-4           [-1, 32, 33, 33]               0
            Conv2d-5            [-1, 1, 33, 33]             801
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.36
Params size (MB): 0.03
Estimated Total Size (MB): 1.40
----------------------------------------------------------------


# Load and build dataloader

In [6]:
with h5py.File('input/train_mscale.h5') as files:
    images = files['data'][:]
    labels = files['label'][:]

images = images.astype(np.float32)
labels = labels.astype(np.float32)

In [7]:
(x_train, x_val, y_train, y_val) = train_test_split(images, labels, test_size=0.2, random_state=2021)

In [8]:
class ResDataset(Dataset):
    def __init__(self, image_data, labels):
        super(ResDataset, self).__init__()
        self.image_data = image_data
        self.labels = labels
            
    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, index):
        image = self.image_data[index]
        label = self.labels[index]

        return {
            'image': torch.tensor(image, dtype=torch.float32),
            'label': torch.tensor(label, dtype=torch.float32)
        }

In [10]:
train_dataset = ResDataset(x_train, y_train)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

val_dataset = ResDataset(x_val, y_val)
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)

# Define training functions

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [34]:
for epoch in tqdm(range(epochs)):
    model.train()
    curr_loss = 0
    curr_psnr = 0
    
    for step, data in tqdm(enumerate(train_dataloader),  total=int(len(train_dataset)/train_dataloader.batch_size)):
        batch_image = data['image'].to(device)
        batch_label = data['label'].to(device)
        
        optimizer.zero_grad()
        logits = model(batch_image)
        loss = criterion(logits, batch_label)
        
        loss.backward()
        optimizer.step()
        
        curr_loss += loss.item()
#         curr_psnr += psnr(batch_label, logits)
    
    final_loss = curr_loss / len(train_dataloader.dataset)
#     final_psnr = curr_psnr / int(len(train_dataset)/train_dataloader.batch_size)
    
    print('Loss: {} at epoch: {}'.format(final_loss, epoch))
#     print('PSNR: {} at epoch: {}'.format(final_psnr, epoch))
    
    # Validation data
    model.eval()
    cur_val_loss = 0
    for data in val_dataloader:
        batch_image = data['image'].to(device)
        batch_label = data['label'].to(device)

        with torch.no_grad():
            logits = model(batch_image)

        loss = criterion(logits, batch_label)
        cur_val_loss += loss.item()

    final_loss = cur_val_loss / len(val_dataloader.dataset)
    print('Validation Loss: {} at epoch: {}'.format(final_loss, epoch))


Loss: 2.7121828863490806e-05 at epoch: 0
Loss: 2.7694796982252244e-05 at epoch: 0



Loss: 2.703065868996046e-05 at epoch: 1
Loss: 2.7683172598993246e-05 at epoch: 1



Loss: 2.703059808518011e-05 at epoch: 2
Loss: 2.7656705328876757e-05 at epoch: 2



Loss: 2.7033520717206035e-05 at epoch: 3
Loss: 2.7844425208678837e-05 at epoch: 3



Loss: 2.7018432627371825e-05 at epoch: 4
Loss: 2.7862734631779266e-05 at epoch: 4


...


Loss: 2.5818049439329196e-05 at epoch: 95
Loss: 2.637952443464037e-05 at epoch: 95



Loss: 2.57633755858803e-05 at epoch: 96
Loss: 2.647910135363966e-05 at epoch: 96



Loss: 2.5791792716633385e-05 at epoch: 97
Loss: 2.6506915147643524e-05 at epoch: 97



Loss: 2.5816786807510317e-05 at epoch: 98
Loss: 2.6341431501096397e-05 at epoch: 98



Loss: 2.5821615212831503e-05 at epoch: 99
Loss: 2.836267341713486e-05 at epoch: 99

